In [1]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('127.0.0.1', 27017)
db = client['test']

In [2]:
db.bar.delete_one({'_id': 123})

db.bar.insert_one({
    '_id' : 123,
    
    'total' : 100,
    'items' : [
        { 'name' : "A", 'price' : 20 },
        { 'name' : "B", 'price' : 50 },
        { 'name' : "C", 'price' : 30 }
    ]
})

In [3]:
pp(db.bar.find_one({}, {'_id': 0}))

{'items': [{'name': 'A', 'price': 20},
           {'name': 'B', 'price': 50},
           {'name': 'C', 'price': 30}],
 'total': 100}


In [4]:
order = 123
product = "A"
price=50

In [33]:
# 1. 移除 特定名字那包
cur = db.bar.update_one({'_id': order, "items.name": { '$eq': product }}, { '$pull': { 'items': { 'name': product }}})

# 2. 更改 total price
total_price = db.bar.aggregate([
    {'$match': {'_id': order}},
    {'$project': {'_id': 0, 'total': {'$sum': '$items.price'}}}
]).next()['total']

db.bar.update_one(
    {'_id' : order},
    {'$set': { "total": total_price}}
)

In [35]:
res = db.bar.update_one({'_id': order, "items.name": { '$ne': product }}, { '$addToSet': { 'items': { 'name': product }}})

if res.modified_count > 0:  # 新增一包 (api的 post作的)
    db.bar.update_one(
        {'_id': order, "items.name": product}, 
        { 
            '$set': { "items.$.price": price},
            '$inc': { 'total' : price }
        }
    )
    
else: # 修改已有的那包 (api的 put作的)
    """
        流程:
        1. 把新增的物件增加價格欄位
        2. 重新計算 total_price
        3. 更新 total欄位 = total_price
    """
    
    # 1.
    db.bar.update_one(
        {'_id': order, "items.name": product}, 
        {'$set': { "items.$.price": price}}
    )
    
    # 2.
    total_price = db.bar.aggregate([
        {'$match': {'_id': order}},
        {'$project': {'_id': 0, 'total': {'$sum': '$items.price'}}}
    ]).next()['total']
    
    # 3.
    db.bar.update_one(
        {'_id': order}, 
        {'$set': { "total": total_price}}
    )